<a href="https://colab.research.google.com/github/luasampaio/data-engineering/blob/main/42_PipelineRDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RDD


Os RDDs podem ser criados de duas maneiras:
- 1.Transformando um RDD existente.
- 2.De um objeto SparkContext ou SparkSession.

### **Quando usar RDDs?**
- Quando você precisa de **controle total** sobre as operações.
- Para dados **não estruturados** ou semi-estruturados.
- Para tarefas que envolvem **operações complexas** que não são suportadas por DataFrames ou Datasets.

- Luciana Sampaio
- 13.03.25

In [16]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .appName('LucianaRDD') \
                    .getOrCreate()

data = [('Luciana','Sampaio','F',30),
       ('Steve','Jobs','M',41),
       ('Bill','Gates','M',62),
]

In [17]:
print(data)

[('Luciana', 'Sampaio', 'F', 30), ('Steve', 'Jobs', 'M', 41), ('Bill', 'Gates', 'M', 62)]


Verificando o tipo

In [18]:
print(type(data))

<class 'list'>


declarar as colunas  e Criando Dataframe

In [19]:
columns = ["PrimeiroNome","Sobrenome","Genero","Salario"]
df = spark.createDataFrame(data=data, schema = columns)
df.show()

+------------+---------+------+-------+
|PrimeiroNome|Sobrenome|Genero|Salario|
+------------+---------+------+-------+
|     Luciana|  Sampaio|     F|     30|
|       Steve|     Jobs|     M|     41|
|        Bill|    Gates|     M|     62|
+------------+---------+------+-------+



Novo Salario

In [20]:
from pyspark.sql.functions import concat_ws,col,lit
df.select(concat_ws(",",df.PrimeiroNome,df.Sobrenome).alias("Nome"), \
        df.Genero,lit(df.Salario*2).alias("novoSalario")).show()

+---------------+------+-----------+
|           Nome|Genero|novoSalario|
+---------------+------+-----------+
|Luciana,Sampaio|     F|         60|
|     Steve,Jobs|     M|         82|
|     Bill,Gates|     M|        124|
+---------------+------+-----------+



In [21]:
print(df.collect())
rdd=df.rdd.map(lambda x:
    (x[0]+","+x[1],x[2],x[3]*2)
    )
df2=rdd.toDF(["Nome","Genero","novoSalario"]   )
df2.show()

[Row(PrimeiroNome='Luciana', Sobrenome='Sampaio', Genero='F', Salario=30), Row(PrimeiroNome='Steve', Sobrenome='Jobs', Genero='M', Salario=41), Row(PrimeiroNome='Bill', Sobrenome='Gates', Genero='M', Salario=62)]
+---------------+------+-----------+
|           Nome|Genero|novoSalario|
+---------------+------+-----------+
|Luciana,Sampaio|     F|         60|
|     Steve,Jobs|     M|         82|
|     Bill,Gates|     M|        124|
+---------------+------+-----------+



In [22]:
# Coleta todos os elementos e retorna como lista
print(rdd.collect())

[('Luciana,Sampaio', 'F', 60), ('Steve,Jobs', 'M', 82), ('Bill,Gates', 'M', 124)]


In [24]:
#Usando função lambda para o novo salario direto
rdd2=df.rdd.map(lambda x:
    (x.PrimeiroNome+","+x.Sobrenome,x.Genero,x.Salario*2)
    )

In [25]:
rdd2.collect()

[('Luciana,Sampaio', 'F', 60),
 ('Steve,Jobs', 'M', 82),
 ('Bill,Gates', 'M', 124)]

Fazendo por Função a transformacao direta de um novo salario

In [27]:
def func1(x):
    PrimeiroNome=x.PrimeiroNome
    Sobrenome=x.Sobrenome
    Nome=PrimeiroNome+","+Sobrenome
    Genero=x.Genero.lower()
    Salario=x.Salario*2
    return (Nome,Genero,Salario)

rdd3=df.rdd.map(lambda x: func1(x))

In [28]:
rdd3.collect()

[('Luciana,Sampaio', 'f', 60),
 ('Steve,Jobs', 'm', 82),
 ('Bill,Gates', 'm', 124)]

In [29]:
def f(x): print(x)
df.rdd.foreach(f)

In [30]:
df.rdd.foreach(lambda x:
    print("Data ==>"+x["PrimeiroNome"]+","+x["Sobrenome"]+","+x["Genero"]+","+str(x["Salario"]*2))
    )

In [32]:
df.collect()

[Row(PrimeiroNome='Luciana', Sobrenome='Sampaio', Genero='F', Salario=30),
 Row(PrimeiroNome='Steve', Sobrenome='Jobs', Genero='M', Salario=41),
 Row(PrimeiroNome='Bill', Sobrenome='Gates', Genero='M', Salario=62)]

Iterando com Collect

In [33]:
dataCollect = df.collect()
for row in dataCollect:
    print(row['PrimeiroNome'] + "," +row['Sobrenome'])

Luciana,Sampaio
Steve,Jobs
Bill,Gates


In [34]:
dataCollect=df.rdd.toLocalIterator()
for row in dataCollect:
    print(row['PrimeiroNome'] + "," +row['Sobrenome'])

Luciana,Sampaio
Steve,Jobs
Bill,Gates


In [35]:
import pandas as pd
pandasDF = df.toPandas()
for index, row in pandasDF.iterrows():
    print(row['PrimeiroNome'] + "," +row['Sobrenome'])

Luciana,Sampaio
Steve,Jobs
Bill,Gates


In [37]:
rdd4=spark.sparkContext.parallelize(data)

In [39]:
rdd4.collect()

[('Luciana', 'Sampaio', 'F', 30),
 ('Steve', 'Jobs', 'M', 41),
 ('Bill', 'Gates', 'M', 62)]

In [40]:
rdd4.count()

3

In [41]:
rdd4.take(1)

[('Luciana', 'Sampaio', 'F', 30)]

In [42]:
rdd5 = rdd4.map(lambda x: (x[0]+","+x[1],x[2],x[3]*2))

In [44]:
rdd5.collect()

[('Luciana,Sampaio', 'F', 60),
 ('Steve,Jobs', 'M', 82),
 ('Bill,Gates', 'M', 124)]

In [46]:
rdd7 = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [48]:
rdd7.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Filtrando com RDD e lambda

In [51]:
rddfiltro = rdd7.filter(lambda x: x  == 8)

In [52]:
rddfiltro.collect()

[8]

In [55]:
## rddfiltro.persist()

In [56]:
rddfiltro.collect()

[8]